##### Project based on the one from datacamp, with olympic medal data compromising records of all events held at the Olympic games between 1896 and 2012. 

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Set plotly offline
init_notebook_mode(connected=True)

In [132]:
import cufflinks as cf

In [133]:
pd.options.display.max_rows = 8 # So i can see .head(4) and .tail(4) in the same cell and with only one comand

In [134]:
medals = pd.read_csv('https://assets.datacamp.com/production/course_1650/datasets/all_medalists.csv')

In [135]:
medals.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


Medals by country

In [136]:
country_names = medals['NOC']

In [137]:
medal_counts = country_names.value_counts()

Which countries have won the most medals?

In [138]:
print(medal_counts.head())

USA    4335
URS    2049
GBR    1594
FRA    1314
ITA    1228
Name: NOC, dtype: int64


How many medals has Portugal (my country) won in the olympics? (Remember this database goes only until 2012)

In [139]:
medal_counts[medal_counts.index == 'POR']

POR    31
Name: NOC, dtype: int64

How many separate bronze, silver and gold medals has each country won?

In [140]:
#Making a pivot table to separate type of medal by countries
counted = medals.pivot_table(aggfunc='count', index='NOC', columns='Medal', values='Athlete')

In [141]:
counted.head()

Medal,Bronze,Gold,Silver
NOC,,,
AFG,1.0,NaN,NaN
AHO,NaN,NaN,1.0
ALG,8.0,4.0,2.0
ANZ,5.0,20.0,4.0
ARG,88.0,68.0,83.0


In [142]:
# Making a new column Total with the sum of the medals
counted['Total'] = counted.sum(axis='columns')

In [143]:
counted.head()

Medal,Bronze,Gold,Silver,Total
NOC,,,,
AFG,1.0,NaN,NaN,1.0
AHO,NaN,NaN,1.0,1.0
ALG,8.0,4.0,2.0,14.0
ANZ,5.0,20.0,4.0,29.0
ARG,88.0,68.0,83.0,239.0


In [144]:
# Sorting values by countries that have won the most medals
counted = counted.sort_values(by='Total', ascending=False)

In [145]:
counted.head()

Medal,Bronze,Gold,Silver,Total
NOC,,,,
USA,1052.0,2088.0,1195.0,4335.0
URS,584.0,838.0,627.0,2049.0
GBR,505.0,498.0,591.0,1594.0
FRA,475.0,378.0,461.0,1314.0
ITA,374.0,460.0,394.0,1228.0


In [146]:
# Reordering the columns
counted = counted[['Gold', 'Silver', 'Bronze', 'Total']]

In [147]:
counted.head()

Medal,Gold,Silver,Bronze,Total
NOC,,,,
USA,2088.0,1195.0,1052.0,4335.0
URS,838.0,627.0,584.0,2049.0
GBR,498.0,591.0,505.0,1594.0
FRA,378.0,461.0,475.0,1314.0
ITA,460.0,394.0,374.0,1228.0


Let's see how my country (Portugal) did for each medal

In [148]:
counted[counted.index == 'POR']

Medal,Gold,Silver,Bronze,Total
NOC,,,,
POR,4.0,9.0,18.0,31.0


Which countries won the most medals in the 1936 Edition in nazi germany?

In [149]:
# Let's take a peak on the main DataFrame again
medals.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
1,Athens,1896,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100m freestyle,M,Silver
2,Athens,1896,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100m freestyle for sailors,M,Bronze
3,Athens,1896,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100m freestyle for sailors,M,Gold
4,Athens,1896,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100m freestyle for sailors,M,Silver


In [150]:
# Making a pivot table to separate Edition by countries
medals_by_edition = medals.pivot_table(aggfunc='count', index='NOC', columns='Edition', values='Athlete')

In [151]:
medals_by_edition.head()

Edition,1896,1900,1904,1908,1912,1920,1924,1928,1932,1936,...,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008
NOC,,,,,,,,,,,,,,,,,,,,,
AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
AHO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
ALG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,2.0,3.0,5.0,NaN,2.0
ANZ,NaN,NaN,NaN,19.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARG,NaN,NaN,NaN,NaN,NaN,NaN,11.0,32.0,4.0,11.0,...,1.0,NaN,NaN,NaN,13.0,2.0,20.0,20.0,47.0,51.0


In [152]:
medals_by_edition.loc[:, 1936].sort_values(ascending=False).head()

NOC
GER    210.0
USA     92.0
ITA     71.0
AUT     50.0
SUI     47.0
Name: 1936, dtype: float64

Do you think that maybe there was home bias for the host country? Let's see how Germany did in terms of medals in the editions just before 1936. 

Medals in the 1932 edition

In [153]:
medals_by_edition.loc['GER', 1932]

38.0

Medals in the 1928 edition

In [154]:
medals_by_edition.loc['GER', 1928]

72.0

###### Cleaning the data

What could be the difference between the 'Event_gender' and 'Gender' columns? You should be able to evaluate your guess by looking
at the unique values of the pairs (Event_gender, Gender) in the data. In particular, you should not see something like
(Event_gender='M', Gender='Women'). However, you will see that, strangely enough, there is an observation with
(Event_gender='W', Gender='Men')

In [155]:
# Selecting both columns for comparison
ev_gen = medals[['Event_gender', 'Gender']]

In [156]:
# Dropping duplicates
ev_gen_uniques = ev_gen.drop_duplicates()

In [157]:
medals_by_gender = medals.groupby(['Event_gender', 'Gender'])

In [158]:
medals_by_gender.head(1)

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
0,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold
348,Paris,1900,Equestrian,Jumping,"VAN DE POELE, Georges",BEL,Men,high jump,X,Bronze
416,Paris,1900,Golf,Golf,"PRATT, Daria",USA,Women,individual,W,Bronze
639,Paris,1900,Tennis,Tennis,"JONES, Marion",ZZX,Women,mixed doubles,X,Bronze
23675,Sydney,2000,Athletics,Athletics,"CHEPCHUMBA, Joyce",KEN,Men,marathon,W,Bronze


In [159]:
medal_count_by_gender = medals_by_gender.count()

In [160]:
medal_count_by_gender.head()

City  Edition  Sport  Discipline  Athlete    NOC  Event  \
Event_gender Gender                                                             
M            Men     20067    20067  20067       20067    20067  20067  20067   
W            Men         1        1      1           1        1      1      1   
             Women    7277     7277   7277        7277     7277   7277   7277   
X            Men      1653     1653   1653        1653     1653   1653   1653   
             Women     218      218    218         218      218    218    218   

                     Medal  
Event_gender Gender         
M            Men     20067  
W            Men         1  
             Women    7277  
X            Men      1653  
             Women     218

We can see the data error with Event_gender 'W' and Gender 'Men.

We will now inspect the suspect record by locating the offending row.

In [161]:
# Creating boolean series for filtering
# Even gender 'W' AND Gender 'Men'
filters = (medals.Event_gender == 'W') & (medals.Gender == 'Men')

# Filtering the dataframe
medals[filters]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
23675,Sydney,2000,Athletics,Athletics,"CHEPCHUMBA, Joyce",KEN,Men,marathon,W,Bronze


In [162]:
# Making a copy of medals to correct (You shouldn't mess with the original data)
medals_clean = medals.copy()

In [163]:
medals_clean.iloc[23675, 6] = 'Women'

In [164]:
medals_clean[filters]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
23675,Sydney,2000,Athletics,Athletics,"CHEPCHUMBA, Joyce",KEN,Women,marathon,W,Bronze


Comparing USA vs USSR during the Cold War

In [165]:
# Filters
during_cold_war = (medals.Edition >= 1952) & (medals.Edition <= 1988)
is_usa_urs = (medals.NOC == 'USA') | (medals.NOC == 'URS')

In [166]:
cold_war_medals = medals[(during_cold_war) & (is_usa_urs)]

In [167]:
cold_war_medals # everything seems ok

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
8019,Helsinki,1952,Aquatics,Diving,"LEE, Samuel",USA,Men,10m platform,M,Gold
8021,Helsinki,1952,Aquatics,Diving,"STOVER-IRWIN, Juno Roslays",USA,Women,10m platform,W,Bronze
8022,Helsinki,1952,Aquatics,Diving,"MCCORMICK, Patricia",USA,Women,10m platform,W,Gold
8023,Helsinki,1952,Aquatics,Diving,"MYERS-POPE, Paula Jean",USA,Women,10m platform,W,Silver
...,...,...,...,...,...,...,...,...,...,...
19587,Seoul,1988,Wrestling,Wrestling Gre-R,"TURLYKHANOV, Daulet",URS,Men,68 - 74kg (welterweight),M,Silver
19589,Seoul,1988,Wrestling,Wrestling Gre-R,"MAMIASHVILI, Mikhail",URS,Men,74 - 82kg (middleweight),M,Gold
19591,Seoul,1988,Wrestling,Wrestling Gre-R,"POPOV, Vladimir",URS,Men,82 - 90kg (light-heavyweight),M,Bronze
19594,Seoul,1988,Wrestling,Wrestling Gre-R,"KOSLOWSKI, Dennis Marvin",USA,Men,90 - 100kg (heavyweight),M,Bronze


In [168]:
country_grouped = cold_war_medals.groupby('NOC')

In [169]:
country_grouped.head()

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
8019,Helsinki,1952,Aquatics,Diving,"LEE, Samuel",USA,Men,10m platform,M,Gold
8021,Helsinki,1952,Aquatics,Diving,"STOVER-IRWIN, Juno Roslays",USA,Women,10m platform,W,Bronze
8022,Helsinki,1952,Aquatics,Diving,"MCCORMICK, Patricia",USA,Women,10m platform,W,Gold
8023,Helsinki,1952,Aquatics,Diving,"MYERS-POPE, Paula Jean",USA,Women,10m platform,W,Silver
...,...,...,...,...,...,...,...,...,...,...
8117,Helsinki,1952,Athletics,Athletics,"JUNK, Bruno",URS,Men,10000m walk,M,Bronze
8135,Helsinki,1952,Athletics,Athletics,"KHNYKINA-DVALISHVILI, Nadezhda",URS,Women,200m,W,Bronze
8140,Helsinki,1952,Athletics,Athletics,"KAZANTSEV, Vladimir",URS,Men,3000m steeplechase,M,Silver
8146,Helsinki,1952,Athletics,Athletics,"LITUEV, Yuri",URS,Men,400m hurdles,M,Silver


In [170]:
# Number of unique sports in which the USA and USSR won medals
Nsports = country_grouped['Sport'].nunique().sort_values(ascending=False)

In [171]:
Nsports

NOC
URS    21
USA    20
Name: Sport, dtype: int64

Counting USA vs. USSR Cold War Olympic Medals

In [172]:
# Medals won by the USA and medals won by the USSR
medals_won_by_country = cold_war_medals.pivot_table(aggfunc='count', index='Edition', columns='NOC', values='Athlete')

In [173]:
medals_won_by_country.loc['Total'] = pd.Series(medals_won_by_country[['USA', 'URS']].sum(), index = ['USA', 'URS'])

In [174]:
medals_won_by_country # The NaN values was because of boycots each country made when the olympics were hosted by the "rival"

NOC,URS,USA
Edition,,
1952,117.0,130.0
1956,169.0,118.0
1960,169.0,112.0
1964,174.0,150.0
...,...,...
1980,442.0,NaN
1984,NaN,333.0
1988,294.0,193.0
Total,2049.0,1495.0


Visualizing USA Medal Counts by Edition

In [175]:
# Create the DataFrame: usa
usa = medals[(medals.NOC == 'USA')]

In [176]:
# Group usa by ['Edition', 'Medal'] and aggregate over 'Athlete'
usa_medals_by_year = usa.groupby(['Edition', 'Medal'])['Athlete'].count()

In [177]:
usa_medals_by_year

Edition  Medal 
1896     Bronze      2
         Gold       11
         Silver      7
1900     Bronze     14
                  ... 
2004     Silver     75
2008     Bronze     81
         Gold      125
         Silver    109
Name: Athlete, Length: 75, dtype: int64

In [178]:
usa_medals_by_year = usa_medals_by_year.unstack(level='Medal')

In [181]:
usa_medals_by_year.head()

Medal,Bronze,Gold,Silver
Edition,,,
1896,2,11,7
1900,14,27,14
1904,111,146,137
1908,15,34,14
1912,31,45,25


In [183]:
#ordering the columns
usa_medals_by_year = usa_medals_by_year[['Gold', 'Silver', 'Bronze']]

In [197]:
usa_medals_by_year.iplot(kind='scatter',
                         yTitle='Number of medals',
                         title='Number of medals won by the USA by edition',
                         color=['gold','darkgray', 'brown'],fill=True)